# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [75]:
import pandas as pd
import numpy as np

df = pd.read_csv('homepage_actions.csv')
df.head()
df.columns
#df.shape
df['action'].unique()
print('Clicks: {}'.format(len(df[df['action'] == 'click'])))
print('No Click: {}'.format(len(df[df['action'] != 'click'])))
print('Views: {}'.format(len(df[df['action'] == 'view'])))
print('Total: {}'.format(len(df)))
# All adds up fine


df.columns
df.id.count()
len(df.id.unique())

# Each visit has an ID.  The ID can have view and click info (2 separate entries)
# lets see if there is overlap???


# These also add up to total.  Looks like the data is good.

df_click = df[df['action'] == 'click']
df_view = df[df['action'] == 'view']

df_view['clicked'] = df_view['id'].isin(list(df_click['id']))


df_view['clicked'].sum()  # THAT WORKED!!

Clicks: 1860
No Click: 6328
Views: 6328
Total: 8188


/Users/aaronlee/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1860

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [76]:
df_exp = df_view[df_view['group'] == 'experiment']
df_ctrl = df_view[df_view['group'] == 'control']

n_exp = len(df_exp)
n_ctrl = len(df_ctrl)

mu_exp = df_exp['clicked'].mean()
mu_ctrl = df_ctrl['clicked'].mean()

print(mu_exp, mu_ctrl)  # first glance looks significant



0.3097463284379172 0.2797118847539016


In [77]:
from scipy import stats

t, p = stats.ttest_ind(df_ctrl['clicked'], df_exp['clicked'], equal_var=False) 
 

# divide by two to get one sided t-test
p / 2

0.004466402814337102

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [78]:
n_exp = len(df_exp)
p_ctrl = df_ctrl['clicked'].mean()
n_exp * mu_ctrl  # expected clicks  using equation above

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [83]:
variance = n_exp * (df_ctrl['clicked'].mean()) * (1 -  df_ctrl['clicked'].mean())  #sqrt[ n * P * ( 1 - P ) ].  # in our case mean is the pval for binomial
variance

sigma = np.sqrt(variance)
sigma

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [87]:
num = df_exp['clicked'].sum() - (n_exp * mu_ctrl) # actual - expected

z = num / sigma
z  # that's my z-score.  Remember, it's just your actual difference divided by sigma.  How many std you are away


p = stats.norm.sf(z)  # survival function
p # p given the zscore


0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

That was a big z score.  
The p-value is small like the other.  They do not differ by a huge amount in comparison to the alpha.  
Both results are similar in that we confidently reject the null hypothesis.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.